In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

import pandas as pd
import seaborn as sns

from sklearn.linear_model import Ridge, Lasso, LinearRegression

import plotly.express as px
import plotly.graph_objects as go

import plotly.offline as py
py.init_notebook_mode(connected=True)

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
df = pd.read_csv('../input/pizza-price-prediction/pizza_v2.csv')

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

<font size="8">**Preprocessing**</font>

In [ ]:
def preprocessing(df):
    df_copy = df.copy()
    
    X = df_copy
    
    X['price_rupiah'] = X.price_rupiah.str.replace('Rp', '')
    X['price_rupiah'] = X.price_rupiah.str.replace(',', '').astype("int")
    X['diameter'] = X.diameter.str.replace(' inch', '').astype("float")
    
    y = X['price_rupiah']
        
    return X,y

In [ ]:
X ,y= preprocessing(df)

<font size="8">**EDA**</font>

In [ ]:
cat_features = X.iloc[:,3:].columns.tolist()
cat_features

In [ ]:
sns.countplot(x="company", data=X)

<font size="5">each company is almost equivalent .</font>

In [ ]:
RP=plt.figure(figsize=(15,18))
for i, feature in enumerate(cat_features):
    r=RP.add_subplot(5,2,i+1)
    sns.countplot(x=feature, data=X)
    r.set_title(feature+" Histogram Plot",color='DarkRed')
RP.tight_layout()  

In [ ]:
RP=plt.figure(figsize=(15,18))
for i, feature in enumerate(cat_features):
    r=RP.add_subplot(5,2,i+1)
    sns.boxplot(x=X[feature], y =X["price_rupiah"])
    r.set_title(feature+" Histogram Plot",color='DarkRed')
RP.tight_layout()  

There are outliners in some datas , but I ignore these values this time , beacause total datas is not much as a whole.
So I keep this data distribution so far .

In [ ]:
fig = px.scatter(X, x="diameter", y="price_rupiah", color='company')
fig.show()

In [ ]:
X.groupby(['company'])['price_rupiah'].mean().sort_values(ascending=False).head(30).plot(kind='bar') 

**<font size="8">LabelEncoding and split the data</font>**

In [ ]:
def label_encoded(feat):
    le = LabelEncoder()
    le.fit(feat)
    return le.transform(feat)

In [ ]:
X['company'] = label_encoded(X['company'])
X['topping'] = label_encoded(X['topping'])
X['variant'] = label_encoded(X['variant'])
X['size'] = label_encoded(X['size'])
X['extra_sauce'] = label_encoded(X['extra_sauce'])
X['extra_cheese'] = label_encoded(X['extra_cheese'])
X['extra_mushrooms'] = label_encoded(X['extra_mushrooms'])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state =42)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**<font size="8">Building a model</font>**

In [ ]:
models=[('LR',LinearRegression()),('DT',DecisionTreeRegressor()),('RF',RandomForestRegressor()),('XGB',XGBRegressor())
        ,('LOG R',LogisticRegression(solver='liblinear')),('SVR',SVR())]
scores=[]

for name,model in models:
    model.fit(X_train,y_train)
    print(model.score(X_test, y_test))
    forest_test_pred=model.predict(X_test)
    forest_train_pred = model.predict(X_train)
    
    score=explained_variance_score(forest_test_pred,y_test)
    scores.append([name,model,model.score(X_test, y_test)])
    
    print('MSE train data: %.3f, MSE test data: %.3f' % (
    mean_squared_error(y_train,forest_train_pred),
    mean_squared_error(y_test,forest_test_pred)))
    print('R2 train data: %.3f, R2 test data: %.3f \n' % (
    r2_score(y_train,forest_train_pred),
    r2_score(y_test,forest_test_pred)))

scores_df=pd.DataFrame(scores,columns=['Name','Model','Score'])
scores_df.sort_values('Score',ascending=False)